In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

import yaml
import pandas as pd

from src.data.load import load_data
from src.data.prepare import prepare_data
from src.models.cv_iterator import leave_last_k

In [2]:
# read config
with open('../config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [3]:
# load and prepare data
dfs = load_data(config=config['data_loader'])
#dfs = prepare_data(dataframes=dfs)

In [4]:
# train-test split
df_train, df_test = leave_last_k(df=dfs['data'], config=config['optimization'])
del df_test

In [5]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

df = df_train[df_train['rating']>=4].copy()
df = df.groupby('user_id')['item_id'].apply(list).tolist()
te = TransactionEncoder()
df = te.fit(df).transform(df)
df = pd.DataFrame(df, columns=te.columns_)

frequent_itemsets = fpgrowth(df, min_support=0.05, use_colnames=True)
del df
frequent_itemsets

,support,itemsets
0,0.502123,(50)
1,0.417197,(100)
2,0.381104,(181)
3,0.358811,(174)
4,0.354565,(127)
...,...,...
203068,0.053079,"(98, 147)"
203069,0.050955,"(56, 147)"
203070,0.052017,"(50, 147, 181)"
203071,0.056263,"(50, 1039)"


In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=2)
rules = rules[rules['antecedents'].apply(lambda x: len(x) == 1)]
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
25,(174),"(98, 50)",0.358811,0.261146,0.200637,0.559172,2.141218,1.0,0.106935,1.676057,0.831231,0.478481,0.403362,0.663732
27,(98),"(100, 174)",0.351380,0.183652,0.139066,0.395770,2.155004,1.0,0.074534,1.351056,0.826314,0.351206,0.259838,0.576498
32,(98),"(50, 100, 174)",0.351380,0.153928,0.117834,0.335347,2.178602,1.0,0.063747,1.272954,0.834063,0.304110,0.214426,0.550432
33,(174),"(98, 100, 50)",0.358811,0.157113,0.117834,0.328402,2.090237,1.0,0.061461,1.255048,0.813466,0.296000,0.203218,0.539201
35,(174),"(98, 181)",0.358811,0.194268,0.158174,0.440828,2.269182,1.0,0.088469,1.440940,0.872305,0.400538,0.306008,0.627518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5669083,(237),(147),0.245223,0.099788,0.057325,0.233766,2.342636,1.0,0.032855,1.174853,0.759337,0.199262,0.148830,0.404117
5669084,(147),(742),0.099788,0.160297,0.050955,0.510638,3.185571,1.0,0.034960,1.715914,0.762136,0.243655,0.417220,0.414260
5669085,(742),(147),0.160297,0.099788,0.050955,0.317881,3.185571,1.0,0.034960,1.319729,0.817056,0.243655,0.242269,0.414260
5669086,(174),(1039),0.358811,0.071125,0.054140,0.150888,2.121434,1.0,0.028620,1.093936,0.824438,0.144068,0.085870,0.456041
